In [46]:
import numpy as np
import pandas as pd
import re

In [47]:
PRICES_STOCK_CSV = "PRICES-STOCK.csv"
PRODUCTS_CSV = "PRODUCTS.csv"

def process_csv_files():
    # Read csv files into separate dataframes
    prices_stock_df = pd.read_csv(PRICES_STOCK_CSV, sep='|'  , engine='python')
    products_df = pd.read_csv(PRODUCTS_CSV, sep='|'  , engine='python')
    
    # Join the prices/stock dataframe and products dataframe
    df_merge = pd.merge(products_df, prices_stock_df, how='right')
    return df_merge

def data_cleaning(dataframe):
    # Clean text
    new_products = dataframe['ITEM_DESCRIPTION'].str.replace("[<p>/]","", regex=True)
    dataframe['ITEM_DESCRIPTION'] = new_products

    # Combine category, sub-category, and sub-sub-category into one category 
    # separated by a pipe symbol in lower case
    dataframe['Category'] = dataframe['CATEGORY'] + ' | ' + dataframe['SUB_CATEGORY'] + ' | ' + dataframe['SUB_SUB_CATEGORY']
    
    # Extract package from item description
    pattern = re.compile(r'(\d+(?!.*\d).*)')
    s = dataframe['ITEM_DESCRIPTION'].str.extract(pattern, expand=True)
    dataframe['Package'] = s
    
    # Format data
    dataframe['Category'] = dataframe['Category'].str.title()
    dataframe['ITEM_NAME'] = dataframe['ITEM_NAME'].str.title()
    dataframe['ITEM_DESCRIPTION'] = dataframe['ITEM_DESCRIPTION'].str.title()
    dataframe['BRAND_NAME'] = dataframe['BRAND_NAME'].str.title()
    
    return dataframe


def select_branch(dataframe, branch1, branch2):
    # Select only MM and RHSM branches that have stock
    filt = ((dataframe['BRANCH'] == branch1) | (dataframe['BRANCH'] == branch2)) & (dataframe['STOCK'] > 0)
    filt_branch_stock = dataframe.loc[filt]
    
    # Create new dataframe with necessary columns for api
    final_df = filt_branch_stock[['EAN', 'BRANCH', 'PRICE', 'STOCK', 'BRAND_NAME', 'Category', 'ITEM_DESCRIPTION', 'ITEM_IMG', 'ITEM_NAME', 'Package', 'SKU']]
    
    return final_df

if __name__ == "__main__":
    merge_df = process_csv_files()
    clean_data = data_cleaning(merge_df)
    results = select_branch(clean_data, 'MM', 'RHSM')
    results.head()


In [48]:
results

,EAN,BRANCH,PRICE,STOCK,BRAND_NAME,Category,ITEM_DESCRIPTION,ITEM_IMG,ITEM_NAME,Package,SKU
1,7501035912854,RHSM,15.3,5,Studio Line,Deli Meat & Cheese | Service Deli | Com Prepar...,Cepillo Dental Infantil Tarzan 0 Pza,https://locahost:8000/images/7501035912854?hei...,Cepillo Dental Infantil Tarzan,0 PZA,223240
15,8410030007788,RHSM,64.2,5,Microlab,Hardlines | Disposable And Birthday Party Supp...,Vino Tinto Maruqes De Penamonte 750 Ml.,https://locahost:8000/images/8410030007788?hei...,Vino Tinto Maruqes De Penamonte,750 ML.,85778
18,33383653006,RHSM,79.5,3,Karcher,Bakery | Bakery | Cakes - In Store,Corazon De Apio 1 Pza,https://locahost:8000/images/33383653006?heigh...,Corazon De Apio,1 PZA,111612
22,40000658928,MM,23.7,5,Altec Lansing,Hardlines | Stationary & Party Goods | Cuadernos,Calcetin Dep Bgn3739 1Un,https://locahost:8000/images/40000658928?heigh...,Calcetin Deportivo,1UN,58179
25,780292000652,RHSM,47.1,2,H2O,Home | Domestic Goods | Toallas Baño,Q Crema Unt Nat 100 100Gr,https://locahost:8000/images/780292000652?heig...,Untable/Fundido,100GR,23522
...,...,...,...,...,...,...,...,...,...,...,...
2296732,5013665113355,RHSM,36.0,3,Sterilite,Hardlines | Books And Gifts | Todo A,Crema 100% De Cacahuates Organica Crujie 227 Grs,https://locahost:8000/images/5013665113355?hei...,Crema 100% De Cacahuates Organica Crujie,227 GRS,133086
2296742,3700906700338,MM,81.9,2,Canada,Consumables | Health And Beauty Aids | Color,Vino Rosado Divine Justine 750 Ml.,https://locahost:8000/images/3700906700338?hei...,Vino Rosado Divine Justine,750 ML.,87273
2296752,7506145820570,MM,49.2,1,Damablanca,Pets | Pets And Supplies | Mascotas - Cuidado ...,Cobertor Home Line Cmfe1802Sb Matrimonia,https://locahost:8000/images/7506145820570?hei...,Cobertor Home Line Cmfe1802Sb Matrimonia,1802SB MATRIMONIA,306369
2296764,7503023492065,RHSM,59.4,3,Baby Shark,Grocery | 92 Grocery | Dessert-Topping Filling...,"Sandalia Simple Fash, 23, Plateado",https://locahost:8000/images/7503023492065?hei...,"Sandalia Simple Fash, 23, Plateado","23, PLATEADO",296517


In [58]:
import json

top_mm_products = results.sort_values(['PRICE', 'BRANCH'], ascending=[0,1]).head(10)

result = top_mm_products.to_json(orient="records")

parsed = json.loads(result)

json.dumps(parsed)  

'[{"EAN": 72649244, "BRANCH": "MM", "PRICE": 90.3, "STOCK": 4, "BRAND_NAME": "Lubritek", "Category": "Hardlines | Sporting Goods | Piscina & Juegos Agua", "ITEM_DESCRIPTION": "Toallita Humeda C30 Pzas 1 Pza", "ITEM_IMG": "https://locahost:8000/images/72649244?height=500&width=500", "ITEM_NAME": "Toallita Humeda C/30 Pzas", "Package": "1 PZA", "SKU": 251261}, {"EAN": 7501325703346, "BRANCH": "MM", "PRICE": 90.3, "STOCK": 5, "BRAND_NAME": "San Blas", "Category": "Grocery | 92 Grocery | Crackers - Salty-Savory", "ITEM_DESCRIPTION": "Salchicha Alpino Para Asar 800 Gr", "ITEM_IMG": "https://locahost:8000/images/7501325703346?height=500&width=500", "ITEM_NAME": "Salchicha Alpino Para Asar 800 Gr", "Package": "800 GR", "SKU": 285805}, {"EAN": 789115006087, "BRANCH": "MM", "PRICE": 90.3, "STOCK": 3, "BRAND_NAME": "Pax", "Category": "Impulse Merchandise | Checkouts/Batteries | Checkouts - Colas Pegamentos & Sellantes", "ITEM_DESCRIPTION": "Acond Super 40 170Ml 170Ml", "ITEM_IMG": "https://locah

In [42]:

final_df

,EAN,BRANCH,PRICE,STOCK,BRAND_NAME,Category,ITEM_DESCRIPTION,ITEM_IMG,ITEM_NAME,Package,SKU
1,7501035912854,RHSM,15.3,5,Studio Line,Deli Meat & Cheese | Service Deli | Com Prepar...,Cepillo Dental Infantil Tarzan 0 Pza,https://locahost:8000/images/7501035912854?hei...,Cepillo Dental Infantil Tarzan,0 PZA,223240
15,8410030007788,RHSM,64.2,5,Microlab,Hardlines | Disposable And Birthday Party Supp...,Vino Tinto Maruqes De Penamonte 750 Ml.,https://locahost:8000/images/8410030007788?hei...,Vino Tinto Maruqes De Penamonte,750 ML.,85778
18,33383653006,RHSM,79.5,3,Karcher,Bakery | Bakery | Cakes - In Store,Corazon De Apio 1 Pza,https://locahost:8000/images/33383653006?heigh...,Corazon De Apio,1 PZA,111612
22,40000658928,MM,23.7,5,Altec Lansing,Hardlines | Stationary & Party Goods | Cuadernos,Calcetin Dep Bgn3739 1Un,https://locahost:8000/images/40000658928?heigh...,Calcetin Deportivo,1UN,58179
25,780292000652,RHSM,47.1,2,H2O,Home | Domestic Goods | Toallas Baño,Q Crema Unt Nat 100 100Gr,https://locahost:8000/images/780292000652?heig...,Untable/Fundido,100GR,23522
...,...,...,...,...,...,...,...,...,...,...,...
2296732,5013665113355,RHSM,36.0,3,Sterilite,Hardlines | Books And Gifts | Todo A,Crema 100% De Cacahuates Organica Crujie 227 Grs,https://locahost:8000/images/5013665113355?hei...,Crema 100% De Cacahuates Organica Crujie,227 GRS,133086
2296742,3700906700338,MM,81.9,2,Canada,Consumables | Health And Beauty Aids | Color,Vino Rosado Divine Justine 750 Ml.,https://locahost:8000/images/3700906700338?hei...,Vino Rosado Divine Justine,750 ML.,87273
2296752,7506145820570,MM,49.2,1,Damablanca,Pets | Pets And Supplies | Mascotas - Cuidado ...,Cobertor Home Line Cmfe1802Sb Matrimonia,https://locahost:8000/images/7506145820570?hei...,Cobertor Home Line Cmfe1802Sb Matrimonia,1802SB MATRIMONIA,306369
2296764,7503023492065,RHSM,59.4,3,Baby Shark,Grocery | 92 Grocery | Dessert-Topping Filling...,"Sandalia Simple Fash, 23, Plateado",https://locahost:8000/images/7503023492065?hei...,"Sandalia Simple Fash, 23, Plateado","23, PLATEADO",296517


In [30]:
PRICES_STOCK_CSV = "PRICES-STOCK.csv"
PRODUCTS_CSV = "PRODUCTS.csv"

prices_stock_df = pd.read_csv(PRICES_STOCK_CSV, sep='|'  , engine='python')
products_df = pd.read_csv(PRODUCTS_CSV, sep='|'  , engine='python')

In [31]:
# Clean text
new_products = products_df['ITEM_DESCRIPTION'].str.replace("[<p>/]","", regex=True)
products_df['ITEM_DESCRIPTION'] = new_products

# Combine category, sub-category, and sub-sub-category into one category 
# separated by a pipe symbol in lower case
products_df['Category'] = products_df['CATEGORY'] + ' | ' + products_df['SUB_CATEGORY'] + ' | ' + products_df['SUB_SUB_CATEGORY']
products_df['Category'] = products_df['Category'].str.title()

In [32]:
import re

pattern = re.compile(r'(\d+(?!.*\d).*)')

s = products_df['ITEM_DESCRIPTION'].str.extract(pattern, expand=True)

products_df['Package'] = s

In [33]:
products_df


,SKU,BUY_UNIT,DESCRIPTION_STATUS,ORGANIC_ITEM,KIRLAND_ITEM,FINELINE_NUMBER,EAN,ITEM_NAME,ITEM_DESCRIPTION,ITEM_IMG,CATEGORY,SUB_CATEGORY,SUB_SUB_CATEGORY,BRAND_NAME,Category,Package
0,52572,UN,B,NaN,False,713115,693177767936,CANASTOS,CANASTO CONEJO F1 A 1UN,https://locahost:8000/images/693177767936?heig...,APPAREL,MENS WEAR,MEN - T-SHIRTS,PUPPY DOG PALS,Apparel | Mens Wear | Men - T-Shirts,1UN
1,278100,NaN,R,NaN,True,345827,399,LIMÓN COLIMA KG,LIMÓN COLIMA KG,https://locahost:8000/images/399?height=500&wi...,ENTERTAINMENT,TOYS & HOBBIES,PELUCHES,ENTRECOT,Entertainment | Toys & Hobbies | Peluches,NaN
2,27404,KG,B,NaN,True,134923,762230099043,CHICLES,TRIDENT 6S SANDIA 9GR,https://locahost:8000/images/762230099043?heig...,HARDLINES,AUTOMOTIVE,APARIENCIA AUTOMOVIL,SPURA,Hardlines | Automotive | Apariencia Automovil,9GR
3,215143,UN,CD,Y,True,773663,7501199416236,JABON LIQUIDO PARA MANOS VITAMINA E,JABON LIQUIDO PARA MANOS VITAMINA E 442 ML.,https://locahost:8000/images/7501199416236?hei...,HARDLINES,SEASONAL,SEASONAL - EASTER,CHAMYTO,Hardlines | Seasonal | Seasonal - Easter,442 ML.
4,85805,KG,CD,N,False,233313,8410113003119,VINO TINTO GRAN CORONAS TORRES 750,VINO TINTO GRAN CORONAS TORRES 750 1 PZA,https://locahost:8000/images/8410113003119?hei...,CONSUMABLES,HOUSEHOLD CHEMICALS,AIR FRESHENER-DEODORIZER,POOPSIE,Consumables | Household Chemicals | Air Freshe...,1 PZA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321537,37542,UN,NaN,N,False,966232,698400300544,LOZA BASICA GRA,TAZA CONSOME 280CC 1UN,https://locahost:8000/images/698400300544?heig...,HARDLINES,HARDWARE,VENTILACION,HO DON CUSTODIO,Hardlines | Hardware | Ventilacion,1UN
321538,199851,NaN,R,Y,True,855034,7501199404301,DETERGENTE LIQUIDO COLOR,DETERGENTE LIQUIDO COLOR 5 LT,https://locahost:8000/images/7501199404301?hei...,GROCERY,DAIRY PRODUCTS,BUTTER-SUBSTITUTES,MADEMSA,Grocery | Dairy Products | Butter-Substitutes,5 LT
321539,144269,NaN,R,Y,True,434058,7501006555752,2 SOBRES PALOMITAS + VALENTINA,2 SOBRES PALOMITAS + VALENTINA 1 PZA,https://locahost:8000/images/7501006555752?hei...,ENTERTAINMENT,ELECTRONICS,EQUIPOS DE AUDIO,DON JUAN,Entertainment | Electronics | Equipos De Audio,1 PZA
321540,312844,KG,B,NaN,True,305744,6971165068775,"CAMISA JORDACHE J, BLANCO, EG/XL","CAMISA JORDACHE J, BLANCO, EGXL",https://locahost:8000/images/6971165068775?hei...,GROCERY,92 GROCERY,MILK-CREAM-SUBSTITUTES-SHELF,MEYER,Grocery | 92 Grocery | Milk-Cream-Substitutes-...,NaN


In [34]:
products_df['ITEM_NAME'] = products_df['ITEM_NAME'].str.title()
products_df['ITEM_DESCRIPTION'] = products_df['ITEM_DESCRIPTION'].str.title()
products_df['BRAND_NAME'] = products_df['BRAND_NAME'].str.title()
products_df

,SKU,BUY_UNIT,DESCRIPTION_STATUS,ORGANIC_ITEM,KIRLAND_ITEM,FINELINE_NUMBER,EAN,ITEM_NAME,ITEM_DESCRIPTION,ITEM_IMG,CATEGORY,SUB_CATEGORY,SUB_SUB_CATEGORY,BRAND_NAME,Category,Package
0,52572,UN,B,NaN,False,713115,693177767936,Canastos,Canasto Conejo F1 A 1Un,https://locahost:8000/images/693177767936?heig...,APPAREL,MENS WEAR,MEN - T-SHIRTS,Puppy Dog Pals,Apparel | Mens Wear | Men - T-Shirts,1UN
1,278100,NaN,R,NaN,True,345827,399,Limón Colima Kg,Limón Colima Kg,https://locahost:8000/images/399?height=500&wi...,ENTERTAINMENT,TOYS & HOBBIES,PELUCHES,Entrecot,Entertainment | Toys & Hobbies | Peluches,NaN
2,27404,KG,B,NaN,True,134923,762230099043,Chicles,Trident 6S Sandia 9Gr,https://locahost:8000/images/762230099043?heig...,HARDLINES,AUTOMOTIVE,APARIENCIA AUTOMOVIL,Spura,Hardlines | Automotive | Apariencia Automovil,9GR
3,215143,UN,CD,Y,True,773663,7501199416236,Jabon Liquido Para Manos Vitamina E,Jabon Liquido Para Manos Vitamina E 442 Ml.,https://locahost:8000/images/7501199416236?hei...,HARDLINES,SEASONAL,SEASONAL - EASTER,Chamyto,Hardlines | Seasonal | Seasonal - Easter,442 ML.
4,85805,KG,CD,N,False,233313,8410113003119,Vino Tinto Gran Coronas Torres 750,Vino Tinto Gran Coronas Torres 750 1 Pza,https://locahost:8000/images/8410113003119?hei...,CONSUMABLES,HOUSEHOLD CHEMICALS,AIR FRESHENER-DEODORIZER,Poopsie,Consumables | Household Chemicals | Air Freshe...,1 PZA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321537,37542,UN,NaN,N,False,966232,698400300544,Loza Basica Gra,Taza Consome 280Cc 1Un,https://locahost:8000/images/698400300544?heig...,HARDLINES,HARDWARE,VENTILACION,Ho Don Custodio,Hardlines | Hardware | Ventilacion,1UN
321538,199851,NaN,R,Y,True,855034,7501199404301,Detergente Liquido Color,Detergente Liquido Color 5 Lt,https://locahost:8000/images/7501199404301?hei...,GROCERY,DAIRY PRODUCTS,BUTTER-SUBSTITUTES,Mademsa,Grocery | Dairy Products | Butter-Substitutes,5 LT
321539,144269,NaN,R,Y,True,434058,7501006555752,2 Sobres Palomitas + Valentina,2 Sobres Palomitas + Valentina 1 Pza,https://locahost:8000/images/7501006555752?hei...,ENTERTAINMENT,ELECTRONICS,EQUIPOS DE AUDIO,Don Juan,Entertainment | Electronics | Equipos De Audio,1 PZA
321540,312844,KG,B,NaN,True,305744,6971165068775,"Camisa Jordache J, Blanco, Eg/Xl","Camisa Jordache J, Blanco, Egxl",https://locahost:8000/images/6971165068775?hei...,GROCERY,92 GROCERY,MILK-CREAM-SUBSTITUTES-SHELF,Meyer,Grocery | 92 Grocery | Milk-Cream-Substitutes-...,NaN


In [35]:
df_merge = pd.merge(products_df, prices_stock_df, how='right')
df_merge

,SKU,BUY_UNIT,DESCRIPTION_STATUS,ORGANIC_ITEM,KIRLAND_ITEM,FINELINE_NUMBER,EAN,ITEM_NAME,ITEM_DESCRIPTION,ITEM_IMG,CATEGORY,SUB_CATEGORY,SUB_SUB_CATEGORY,BRAND_NAME,Category,Package,BRANCH,PRICE,STOCK
0,142308,NaN,A,NaN,False,598980,5060108450089,Agua Tonica Ginger,Agua Tonica Ginger 200 Ml.,https://locahost:8000/images/5060108450089?hei...,DELI MEAT & CHEESE,LUNCH MEAT,DULCE,Betty Crocker,Deli Meat & Cheese | Lunch Meat | Dulce,200 ML.,HRO,45.6,3
1,223240,UN,B,NaN,False,786651,7501035912854,Cepillo Dental Infantil Tarzan,Cepillo Dental Infantil Tarzan 0 Pza,https://locahost:8000/images/7501035912854?hei...,DELI MEAT & CHEESE,SERVICE DELI,COM PREPARADAS,Studio Line,Deli Meat & Cheese | Service Deli | Com Prepar...,0 PZA,RHSM,15.3,5
2,49514,KG,CD,NaN,True,607009,775512804491,Juegos De Mesa,Puzzle 9 Piezas 1Un,https://locahost:8000/images/775512804491?heig...,APPAREL,LADIES WEAR,LADY - JUVENIL CASUAL,Jovan,Apparel | Ladies Wear | Lady - Juvenil Casual,1UN,RHSM,56.7,-2
3,111807,KG,R,NaN,False,420531,898685001166,Jugo,Jugo 946 Ml.,https://locahost:8000/images/898685001166?heig...,HARDLINES,PAINT AND ACCESSORIES,PINTURA - EXTERIOR,Underskin By Top,Hardlines | Paint And Accessories | Pintura - ...,946 ML.,RHSM,48.0,-4
4,223543,NaN,CD,NaN,False,379448,7509552913170,Crema Facial 3 En1 Aclarante,Crema Facial 3 En1 Aclarante 200 Ml.,https://locahost:8000/images/7509552913170?hei...,GROCERY,92 GROCERY,SUGAR-SUBSTITUTE,Ocean Spray,Grocery | 92 Grocery | Sugar-Substitute,200 ML.,MS,8.7,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2296762,243017,UN,NaN,Y,True,94622,75486087999,Bronceador Fps 30 Zanahoria Sray,Bronceador Fps 30 Zanahoria Sray 180 Ml.,https://locahost:8000/images/75486087999?heigh...,HOME,FABRIC & CRAFTS,FLORAL - ACCESORIOS,Dermovagisil,Home | Fabric & Crafts | Floral - Accesorios,180 ML.,HRO,73.2,4
2296763,185713,UN,CD,N,False,546547,735257001493,Gelatina De Agua Sabor Cereza 140 Gr,Gelatina De Agua Sabor Cereza 140 Gr 140 Grs,https://locahost:8000/images/735257001493?heig...,ENTERTAINMENT,ELECTRICAL APPLIANCES,CALEFACCION,Natura,Entertainment | Electrical Appliances | Calefa...,140 GRS,MS,85.5,2
2296764,296517,UN,A,N,True,892065,7503023492065,"Sandalia Simple Fash, 23, Plateado","Sandalia Simple Fash, 23, Plateado",https://locahost:8000/images/7503023492065?hei...,GROCERY,92 GROCERY,DESSERT-TOPPING FILLING BASE,Baby Shark,Grocery | 92 Grocery | Dessert-Topping Filling...,"23, PLATEADO",RHSM,59.4,3
2296765,52814,NaN,B,N,True,335651,780220012903,Malva,Marshmallow Trenza 250Gr,https://locahost:8000/images/780220012903?heig...,BAKERY,COMMERCIAL BREAD,BREAD SHELF STABLE,Princesita Sofia,Bakery | Commercial Bread | Bread Shelf Stable,250GR,MS,23.7,-1


In [36]:
# Select only MM and RHSM branches that have stock
filt = ((df_merge['BRANCH'] == 'MM') | (df_merge['BRANCH'] == 'RHSM')) & (df_merge['STOCK'] > 0)
filt_branch_stock = df_merge.loc[filt]
filt_branch_stock

,SKU,BUY_UNIT,DESCRIPTION_STATUS,ORGANIC_ITEM,KIRLAND_ITEM,FINELINE_NUMBER,EAN,ITEM_NAME,ITEM_DESCRIPTION,ITEM_IMG,CATEGORY,SUB_CATEGORY,SUB_SUB_CATEGORY,BRAND_NAME,Category,Package,BRANCH,PRICE,STOCK
1,223240,UN,B,NaN,False,786651,7501035912854,Cepillo Dental Infantil Tarzan,Cepillo Dental Infantil Tarzan 0 Pza,https://locahost:8000/images/7501035912854?hei...,DELI MEAT & CHEESE,SERVICE DELI,COM PREPARADAS,Studio Line,Deli Meat & Cheese | Service Deli | Com Prepar...,0 PZA,RHSM,15.3,5
15,85778,NaN,CD,Y,True,859191,8410030007788,Vino Tinto Maruqes De Penamonte,Vino Tinto Maruqes De Penamonte 750 Ml.,https://locahost:8000/images/8410030007788?hei...,HARDLINES,DISPOSABLE AND BIRTHDAY PARTY SUPPLIES,SUMINISTRO FIESTAS,Microlab,Hardlines | Disposable And Birthday Party Supp...,750 ML.,RHSM,64.2,5
18,111612,KG,R,Y,False,377418,33383653006,Corazon De Apio,Corazon De Apio 1 Pza,https://locahost:8000/images/33383653006?heigh...,BAKERY,BAKERY,CAKES - IN STORE,Karcher,Bakery | Bakery | Cakes - In Store,1 PZA,RHSM,79.5,3
22,58179,UN,A,N,True,103112,40000658928,Calcetin Deportivo,Calcetin Dep Bgn3739 1Un,https://locahost:8000/images/40000658928?heigh...,HARDLINES,STATIONARY & PARTY GOODS,CUADERNOS,Altec Lansing,Hardlines | Stationary & Party Goods | Cuadernos,1UN,MM,23.7,5
25,23522,KG,R,N,True,652595,780292000652,Untable/Fundido,Q Crema Unt Nat 100 100Gr,https://locahost:8000/images/780292000652?heig...,HOME,DOMESTIC GOODS,TOALLAS BAÑO,H2O,Home | Domestic Goods | Toallas Baño,100GR,RHSM,47.1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2296732,133086,NaN,CD,Y,False,608556,5013665113355,Crema 100% De Cacahuates Organica Crujie,Crema 100% De Cacahuates Organica Crujie 227 Grs,https://locahost:8000/images/5013665113355?hei...,HARDLINES,BOOKS AND GIFTS,TODO A,Sterilite,Hardlines | Books And Gifts | Todo A,227 GRS,RHSM,36.0,3
2296742,87273,NaN,B,Y,True,511829,3700906700338,Vino Rosado Divine Justine,Vino Rosado Divine Justine 750 Ml.,https://locahost:8000/images/3700906700338?hei...,CONSUMABLES,HEALTH AND BEAUTY AIDS,COLOR,Canada,Consumables | Health And Beauty Aids | Color,750 ML.,MM,81.9,2
2296752,306369,KG,CD,N,True,555864,7506145820570,Cobertor Home Line Cmfe1802Sb Matrimonia,Cobertor Home Line Cmfe1802Sb Matrimonia,https://locahost:8000/images/7506145820570?hei...,PETS,PETS AND SUPPLIES,MASCOTAS - CUIDADO - HIGIENE,Damablanca,Pets | Pets And Supplies | Mascotas - Cuidado ...,1802SB MATRIMONIA,MM,49.2,1
2296764,296517,UN,A,N,True,892065,7503023492065,"Sandalia Simple Fash, 23, Plateado","Sandalia Simple Fash, 23, Plateado",https://locahost:8000/images/7503023492065?hei...,GROCERY,92 GROCERY,DESSERT-TOPPING FILLING BASE,Baby Shark,Grocery | 92 Grocery | Dessert-Topping Filling...,"23, PLATEADO",RHSM,59.4,3


In [37]:
final_df = filt_branch_stock[['EAN', 'BRANCH', 'PRICE', 'STOCK', 'BRAND_NAME', 'Category', 'ITEM_DESCRIPTION', 'ITEM_IMG', 'ITEM_NAME', 'Package', 'SKU']]
final_df

,EAN,BRANCH,PRICE,STOCK,BRAND_NAME,Category,ITEM_DESCRIPTION,ITEM_IMG,ITEM_NAME,Package,SKU
1,7501035912854,RHSM,15.3,5,Studio Line,Deli Meat & Cheese | Service Deli | Com Prepar...,Cepillo Dental Infantil Tarzan 0 Pza,https://locahost:8000/images/7501035912854?hei...,Cepillo Dental Infantil Tarzan,0 PZA,223240
15,8410030007788,RHSM,64.2,5,Microlab,Hardlines | Disposable And Birthday Party Supp...,Vino Tinto Maruqes De Penamonte 750 Ml.,https://locahost:8000/images/8410030007788?hei...,Vino Tinto Maruqes De Penamonte,750 ML.,85778
18,33383653006,RHSM,79.5,3,Karcher,Bakery | Bakery | Cakes - In Store,Corazon De Apio 1 Pza,https://locahost:8000/images/33383653006?heigh...,Corazon De Apio,1 PZA,111612
22,40000658928,MM,23.7,5,Altec Lansing,Hardlines | Stationary & Party Goods | Cuadernos,Calcetin Dep Bgn3739 1Un,https://locahost:8000/images/40000658928?heigh...,Calcetin Deportivo,1UN,58179
25,780292000652,RHSM,47.1,2,H2O,Home | Domestic Goods | Toallas Baño,Q Crema Unt Nat 100 100Gr,https://locahost:8000/images/780292000652?heig...,Untable/Fundido,100GR,23522
...,...,...,...,...,...,...,...,...,...,...,...
2296732,5013665113355,RHSM,36.0,3,Sterilite,Hardlines | Books And Gifts | Todo A,Crema 100% De Cacahuates Organica Crujie 227 Grs,https://locahost:8000/images/5013665113355?hei...,Crema 100% De Cacahuates Organica Crujie,227 GRS,133086
2296742,3700906700338,MM,81.9,2,Canada,Consumables | Health And Beauty Aids | Color,Vino Rosado Divine Justine 750 Ml.,https://locahost:8000/images/3700906700338?hei...,Vino Rosado Divine Justine,750 ML.,87273
2296752,7506145820570,MM,49.2,1,Damablanca,Pets | Pets And Supplies | Mascotas - Cuidado ...,Cobertor Home Line Cmfe1802Sb Matrimonia,https://locahost:8000/images/7506145820570?hei...,Cobertor Home Line Cmfe1802Sb Matrimonia,1802SB MATRIMONIA,306369
2296764,7503023492065,RHSM,59.4,3,Baby Shark,Grocery | 92 Grocery | Dessert-Topping Filling...,"Sandalia Simple Fash, 23, Plateado",https://locahost:8000/images/7503023492065?hei...,"Sandalia Simple Fash, 23, Plateado","23, PLATEADO",296517
